In [1]:
import csv
from mysql.connector import MySQLConnection, Error
from configparser import ConfigParser
from pandas import DataFrame
import pandas as pd

In [2]:
def read_config(config_file = 'config.ini', section = 'mysql'):
    parser = ConfigParser()
    parser.read(config_file)
    
    config = {}
    
    if parser.has_section(section):
        # Parse the configuration file.
        items = parser.items(section)
        
        # Construct the parameter dictionary.
        for item in items:
            config[item[0]] = item[1]
            
    else:
        raise Exception(f'Section [{section}] missing ' + \
                        f'in config file {filename}')
    
    return config

In [3]:
db_config = read_config() # Calling read config function

In [4]:
def make_connection(config_file = 'config.ini', section = 'mysql'): #function to create connection
    try:
        db_config = read_config(config_file, section)            
        conn = MySQLConnection(**db_config)
        
        if conn.is_connected():
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [5]:
conn = make_connection() # calling connection function
conn

In [6]:
cursor = conn.cursor()

In [7]:
def do_query_return_all(conn, sql): # function to execte the query
    cursor = None
    
    try:
        # Do the query.
        cursor = conn.cursor()
        cursor.execute(sql);

        # Return the fetched data as a list of tuples,
        # one tuple per table row.
        rows = cursor.fetchall()
        count = cursor.rowcount

        cursor.close()
        return [rows, count]

    except Error as e:
        print('Query failed')
        print(e)

        cursor.close()
        return [(), 0]

### use SQL INSERT INTO commands to populate each table with around 5 rows of example data.
#### Here I have used INSERT statement for 4 tables and the data was populated from .csv file.

In [8]:
sql_student = ( " INSERT INTO CLASS.Student ( StudentID, FirstName, LastName, Gender) \n"
    +   "VALUES (%s, %s, %s, %s)"
       )   

        
first = True
with open('Student.csv', newline='') as csv_file:  
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            cursor.execute(sql_student, row)
        first = False



In [9]:
sql_studentcourses = ( " INSERT INTO CLASS.StudentCourses ( StudentID, CourseTitle, Grade) \n"
    +   "VALUES (%s, %s, %s)"            
       ) 

               
first = True
with open('StudentCourses.csv', newline='') as csv_file:  
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            cursor.execute(sql_studentcourses, row)
        first = False    

 

In [10]:
sql_teacher = ( " INSERT INTO CLASS.Teacher( TeacherID, TeacherName) \n"
    +   "VALUES (%s, %s)"                    
       )  
first = True
with open('Teacher.csv', newline='') as csv_file:  
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            cursor.execute(sql_teacher, row)
        first = False

In [11]:
sql_courses = ( " INSERT INTO CLASS.Courses ( CourseTitle, Fee, Qualification, TeacherID) \n"
    +   "VALUES (%s, %s, %s, %s)"
              )

first = True
with open('Courses.csv', newline='') as csv_file:  
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            cursor.execute(sql_courses, row)
        first = False 

In [12]:
sql3 = ( "ALTER TABLE CLASS.student ADD BusPass varchar(5) \n"

       )
print(sql3)
rows, count = do_query_return_all(conn, sql3)

ALTER TABLE CLASS.student ADD BusPass varchar(5) 



### use SQL SELECT commands to print the contents of each table. 

In [13]:
sql_display = ( "SELECT * FROM CLASS.Student")
rows, count = do_query_return_all(conn, sql_display)
data=DataFrame(rows)
data.columns=['StudentID', 'FirstName', 'LastName', 'Gender','Email']
data

,StudentID,FirstName,LastName,Gender,Email
0,101,Rajeev,Yenugula,Male,None
1,102,Kishore,Ruvva,Male,None
2,103,Hruthik,Vinnakota,Male,None
3,104,Sahanti,Zade,Female,None
4,105,Saavn,Beli,Male,None


In [14]:
sql_display = ( "SELECT * FROM CLASS.StudentCourses")
rows, count = do_query_return_all(conn, sql_display)
data=DataFrame(rows)
data.columns=['StudentID', 'Course Titles', 'Grade']
data

,StudentID,Course Titles,Grade
0,101,220,A
1,101,225,B
2,101,230,C
3,102,200,A
4,102,220,A
5,102,225,C
6,102,230,B
7,103,220,B
8,103,230,A
9,103,235,A


In [15]:
sql_display = ( "SELECT * FROM CLASS.Courses")
rows, count = do_query_return_all(conn, sql_display)
data=DataFrame(rows)
data.columns=['CourseTitle', 'Fee',' Qualification', 'TeacherID']
data

,CourseTitle,Fee,Qualification,TeacherID
0,200,"$2,350",Certificate,4
1,220,"$2,200",Advanced Level,1
2,225,"$2,300",Advanced Level,2
3,230,"$2,250",Diploma,3
4,235,"$2,200",Advanced Level,5
5,240,"$2,300",Advanced Level,6
6,245,"$2,250",Advanced Level,7
7,255,"$2,200",Certificate,8


In [16]:
sql_display = ( "SELECT * FROM CLASS.Teacher")
rows, count = do_query_return_all(conn, sql_display)
data=DataFrame(rows)
data.columns=[ 'TeacherID', 'Teacher Name']
data

,TeacherID,Teacher Name
0,1,Bond
1,2,Steve
2,3,David
3,4,Kim
4,5,Southee
5,6,Starn
6,7,Broad


In [17]:
cursor.close()
conn.close()